<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## downloading essential modules

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Using the cs-en translation

In [3]:
from datasets import load_dataset

import torch
from transformers import MarianMTModel, MarianTokenizer



In [4]:
ds = load_dataset("wmt/wmt16", "tr-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/153k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205756 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [6]:
ds['train'][0]

{'translation': {'en': "Kosovo's privatisation process is under scrutiny",
  'tr': "Kosova'nın özelleştirme süreci büyüteç altında"}}

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tr-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tr-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [9]:
# https://huggingface.co/docs/transformers/model_doc/marian

In [10]:
def generate_translation(src_text, max_length = 50):

  # Tokenize input
  encoder_inputs = tokenizer(src_text, return_tensors="pt")

  # intializes the decoder input with decoder start token
  decoder_input = torch.tensor([[model.config.decoder_start_token_id]])

  # change the model to eval mode.
  model.eval()
  with torch.no_grad():

    generated_tokens = []

    while len(generated_tokens) < max_length:

      outputs = model(
          input_ids=encoder_inputs.input_ids,
          attention_mask=encoder_inputs.attention_mask,
          decoder_input_ids=decoder_input
      )


      next_token_logits = outputs.logits[:, -1, :]


      # get the token with maximum logits value
      next_token = torch.argmax(next_token_logits, dim=-1)

      generated_tokens.append(next_token.item())

      if next_token.item() == tokenizer.eos_token_id:
          break
      print(decoder_input)
      decoder_input = torch.cat([decoder_input, next_token.unsqueeze(0)], dim=1)
      print(decoder_input)
    translated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
  return translated_text


In [11]:

### print("\nStep by step generation:")
# for i in range(len(generated_tokens)):
#     partial_text = tokenizer.decode(generated_tokens[:i+1], skip_special_tokens=True)
#     print(f"Step {i+1}: {partial_text}")

In [12]:
model.config.num_beams

6

In [13]:
print(generate_translation(ds['validation'][1]['translation']['tr']))

tensor([[62388]])
tensor([[62388, 31058]])
tensor([[62388, 31058]])
tensor([[62388, 31058,     4]])
tensor([[62388, 31058,     4]])
tensor([[62388, 31058,     4,     8]])
tensor([[62388, 31058,     4,     8]])
tensor([[62388, 31058,     4,     8,  1129]])
tensor([[62388, 31058,     4,     8,  1129]])
tensor([[62388, 31058,     4,     8,  1129,  1782]])
tensor([[62388, 31058,     4,     8,  1129,  1782]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9, 11363]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9, 11363]])
tensor([[62388, 31058,     4

In [14]:
ds["validation"][1]["translation"]["tr"]

"Norveç'in beş milyon insanı en yüksek yaşam standartlarının tadını çıkarıyor, sadece Avrupa'da değil, dünyada."

In [15]:
ds["validation"][1]["translation"]["en"]

"Norway's five million people enjoy one of the highest standards of living, not just in Europe, but in the world."

In [16]:
# reference to get the python file from github



https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/A.02-Downloading_Python_source_files_from_github.ipynb

In [17]:
# get constraints

user = "vvikasreddy"
repo = "lexically_constrained_beam_search_"
pyfile = "constraints.py"

# url = "https://github.com/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.py"
url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

import constraints

--2024-11-04 13:20:52--  https://raw.githubusercontent.com/vvikasreddy/lexically_constrained_beam_search_/main/constraints.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4406 (4.3K) [text/plain]
Failed to rename constraints.py to constraints.py.1: (2) No such file or directory
Saving to: ‘constraints.py’

constraints.py      100%[===================>]   4.30K  --.-KB/s    in 0s      

2024-11-04 13:20:53 (56.6 MB/s) - ‘constraints.py’ saved [4406/4406]



In [18]:
# takes nice 5 minutes to get all the constraints of length 2
c = constraints.get_constraints()

In [144]:
def generate_translation(src_text, decoder_input = [], probabilites = [],  max_length = 1, get_constrained_token_probability = -1):

  """generate the decoder input ids and corresponding probabilities"""


  # Tokenize input
  encoder_inputs = tokenizer(src_text, return_tensors="pt")

  print(decoder_input, "sub initial")
  # initialize if not initialized.
  if decoder_input == []:
    # because the intial decoder start token has probabilitty 1
    probabilites = torch.tensor([[1]])
    decoder_input = torch.tensor([[model.config.decoder_start_token_id]])

  print(decoder_input, "inital")
  # change the model to eval mode.
  model.eval()
  with torch.no_grad():

    generated_tokens = []

    outputs = model(
        input_ids=encoder_inputs.input_ids,
        attention_mask=encoder_inputs.attention_mask,
        decoder_input_ids=decoder_input
    )

    next_token_logits = outputs.logits[:, -1, :]

    if get_constrained_token_probability != -1:
      # print(next_token_logits.shape)
      softmax_  = torch.softmax(next_token_logits, dim=-1)
      return softmax_[0][get_constrained_token_probability]

    # get the token with maximum logits value

    next_token = torch.argmax(next_token_logits, dim=-1)


    top_probs, top_indices = torch.topk(torch.softmax(next_token_logits, dim=-1), k = 5)


  decoder_inputs = []
  probs = []
  # to visualize data
  vis_data = []
  print(top_indices)
  for indx, id in enumerate(top_indices[0]):
    # print(decoder_input, id.unsqueeze(0).unsqueeze(0), id, "jhere")
    # print(probabilities, top_probs[0][indx].unsqueeze(0).unsqueeze(0))
    decoder_inputs.append(torch.cat([decoder_input, id.unsqueeze(0).unsqueeze(0)], dim=1))
    probs.append(torch.cat([probabilites, top_probs[0][indx].unsqueeze(0).unsqueeze(0)], dim=1))
    # print(decoder_inputs[indx], indx)
    vis_data.append((vis_data, tokenizer.decode(decoder_inputs[indx].squeeze(), skip_special_tokens = True)))
  # translated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
  print(decoder_inputs, probs)
  # print(vis_data)
  return decoder_inputs, probs, vis_data
  # return decoder_input, probabilities


In [146]:
print(generate_translation(ds['validation'][1]['translation']['tr'], decoder_input = torch.tensor([[62388,  1969]]), probabilites = torch.tensor([[0.0000, 0.0000]])))
# c

tensor([[62388,  1969]]) sub initial
tensor([[62388,  1969]]) inital
tensor([[261,  15, 510,   8,  47]])
[tensor([[62388,  1969,   261]]), tensor([[62388,  1969,    15]]), tensor([[62388,  1969,   510]]), tensor([[62388,  1969,     8]]), tensor([[62388,  1969,    47]])] [tensor([[0.0000, 0.0000, 0.3950]]), tensor([[0.0000, 0.0000, 0.1113]]), tensor([[0.0000, 0.0000, 0.0247]]), tensor([[0.0000, 0.0000, 0.0110]]), tensor([[0.0000, 0.0000, 0.0105]])]
([tensor([[62388,  1969,   261]]), tensor([[62388,  1969,    15]]), tensor([[62388,  1969,   510]]), tensor([[62388,  1969,     8]]), tensor([[62388,  1969,    47]])], [tensor([[0.0000, 0.0000, 0.3950]]), tensor([[0.0000, 0.0000, 0.1113]]), tensor([[0.0000, 0.0000, 0.0247]]), tensor([[0.0000, 0.0000, 0.0110]]), tensor([[0.0000, 0.0000, 0.0105]])], [([...], 'Koso'), ([...], 'Kost'), ([...], 'Kosum'), ([...], 'Koss'), ([...], 'Kosa')])


In [74]:
def get_ngrams(src, n = 2, ):
  src = src.split(" ")

  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]

  return src

def constraints_tokens(src, c):
  ngrams = get_ngrams(src)
  constraints_src = []
  for ngram in ngrams:
    # print(ngram)
    if ngram in c:
      for gram in ngram:

        constraints_src.append(tokenizer(gram, return_tensors="pt"))
  return constraints_src

def get_input_ids(data):

  input_ids = []
  for example in data:
    input_ids.append((example['input_ids'][0].tolist())[0])
  return input_ids

In [75]:
x = constraints_tokens(ds['validation'][1]['translation']['tr'] + ' Times' +  ' için', c)
# c
print(x)
print(get_input_ids(x))

[{'input_ids': tensor([[3762,    0]]), 'attention_mask': tensor([[1, 1]])}, {'input_ids': tensor([[37,  0]]), 'attention_mask': tensor([[1, 1]])}]
[3762, 0]
[37, 0]
[3762, 37]


In [155]:
def beam_search(maxlen, numC, k, src, constraints):

    decoder_start_token = model.config.decoder_start_token_id

    # initialize the grids
    grids = [[[] for _ in range(numC)] for _ in range(maxlen)]
    # should the below be the size
    # grids = [[[] for _ in range(numC)] for _ in range(maxlen)]

    # intialize the first grid to start hyp
    grids[0][0] = [1]

    # remove during test
    # constrained_tokens = get_input_ids(constraints_tokens(src, constraints))
    # temporary
    constrained_tokens = [3762, 37]

    generated_constraint_index = 0

    print(grids)
    grids[0][0]

    for t in range(1, maxlen):


      index_c = max(0, (numC - t) - maxlen)

      for c in range(index_c, min(t, numC) + 1):


          print("cur iteration ", t, c )
          s = []
          g = []

          print(grids[t-1][c], len(grids[t-1][c]), "print the gris")

          print(grids[t-1][c], "printing the grid elements ")
          for element in grids[t-1][c]:

            # if element == 'start':
            # guess ther is no need for conditioning, just generate.
            print(element)
            if type(element) == int:
              decoder_input = []
              probs =[]
            else:
              decoder_input = element[0]
              probs = element[1]

            print(element)
            print(decoder_input, "di")
            print(probs, "p")
            g, probs, vis_data = generate_translation(src_text= src, decoder_input = decoder_input, probabilites = probs)
            print(g, "in the first for ")


          # retrieve the  probability of the constraint and add that to the decoder_input.
          if c > 0 and constrained_tokens:
            for element in grids[t-1][c-1]:

              print(t-1,c-1, "tc", constrained_tokens[c-1], element, constrained_tokens, "hgello")
              # get the constraints store them
              cons = generate_translation(src, decoder_input = [] if element ==1 else element, get_constrained_token_probability = constrained_tokens[c - 1])
              print([cons])

              if c == 1:
                decoder_inputs = torch.tensor([[model.config.decoder_start_token_id]])
                prob = torch.tensor([[1]])

              decoder_inputs = torch.cat([decoder_inputs, torch.tensor(constrained_tokens[c-1]).unsqueeze(0).unsqueeze(0)], dim=1)
              prob = torch.cat([prob, torch.tensor(cons).unsqueeze(0).unsqueeze(0)], dim=1)

              # decoder_input = torch.tensor(constrained_tokens[c-1]).unsqueeze(0).unsqueeze(0)
              # probs = torch.tensor(cons).unsqueeze(0).unsqueeze(0)
              print(decoder_inputs, "-------------------")
              print(probs)

              print(type(g), type(probs))
              g.append(decoder_inputs)

              probs.append(prob)

              print(g, '==========')
              print(probs)
              # s, probs, vis_data = generate_translation(src_text= src, probabilites = probs, decoder_input = decoder_inputs)
              print(s)


          print()
          # combine g and s properly and get the top k values.
          # grid[t][c] = argmax(g | s)

          # print(g)
          # print(probs)

          # print(vis_data)
          # print(len(g[0]))
          print(g)
          print(probs)
          print(vis_data)
          print(t,c, "t,c")
          grids[t][c] = (g, probs)
          print(grids[t][c])
          print('asd-------------------------------------------')

  # sanity : print grids
    print(grids)


beam_search(maxlen= 4, numC=2, k = 5, src = ds["train"][1]["translation"]["tr"], constraints = c)

[[[1], []], [[], []], [[], []], [[], []]]
cur iteration  1 0
[1] 1 print the gris
[1] printing the grid elements 
1
1
[] di
[] p
[] sub initial
tensor([[62388]]) inital
tensor([[1969,  323,   67, 1132,  626]])
[tensor([[62388,  1969]]), tensor([[62388,   323]]), tensor([[62388,    67]]), tensor([[62388,  1132]]), tensor([[62388,   626]])] [tensor([[1.0000, 0.8746]]), tensor([[1.0000, 0.0156]]), tensor([[1.0000, 0.0114]]), tensor([[1.0000, 0.0039]]), tensor([[1.0000, 0.0038]])]
[tensor([[62388,  1969]]), tensor([[62388,   323]]), tensor([[62388,    67]]), tensor([[62388,  1132]]), tensor([[62388,   626]])] in the first for 

[tensor([[62388,  1969]]), tensor([[62388,   323]]), tensor([[62388,    67]]), tensor([[62388,  1132]]), tensor([[62388,   626]])]
[tensor([[1.0000, 0.8746]]), tensor([[1.0000, 0.0156]]), tensor([[1.0000, 0.0114]]), tensor([[1.0000, 0.0039]]), tensor([[1.0000, 0.0038]])]
[([...], 'Kos'), ([...], 'In'), ([...], 'He'), ([...], 'With'), ([...], 'As')]
1 0 t,c
([tensor(

<ipython-input-155-6f5c33140e38>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prob = torch.cat([prob, torch.tensor(cons).unsqueeze(0).unsqueeze(0)], dim=1)


tensor([[ 261, 2922,   47, 1484, 3339]])
[tensor([[62388,  1969,   261]]), tensor([[62388,  1969,  2922]]), tensor([[62388,  1969,    47]]), tensor([[62388,  1969,  1484]]), tensor([[62388,  1969,  3339]])] [tensor([[6.2388e+04, 3.2300e+02, 9.2845e-01]]), tensor([[6.2388e+04, 3.2300e+02, 1.8652e-03]]), tensor([[6.2388e+04, 3.2300e+02, 1.4004e-03]]), tensor([[6.2388e+04, 3.2300e+02, 5.9243e-04]]), tensor([[6.2388e+04, 3.2300e+02, 2.9697e-04]])]
[tensor([[62388,  1969,   261]]), tensor([[62388,  1969,  2922]]), tensor([[62388,  1969,    47]]), tensor([[62388,  1969,  1484]]), tensor([[62388,  1969,  3339]])] in the first for 
[tensor([[1.0000, 0.8746]]), tensor([[1.0000, 0.0156]]), tensor([[1.0000, 0.0114]]), tensor([[1.0000, 0.0039]]), tensor([[1.0000, 0.0038]]), tensor([[1.0000e+00, 2.5136e-06]])]
[tensor([[1.0000, 0.8746]]), tensor([[1.0000, 0.0156]]), tensor([[1.0000, 0.0114]]), tensor([[1.0000, 0.0039]]), tensor([[1.0000, 0.0038]]), tensor([[1.0000e+00, 2.5136e-06]])]
tensor([[1.000

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)